In [100]:
import requests
import json
from mlProject.utils.ura_token import ura_token
from pandas import json_normalize
import pandas as pd
import psycopg2
import uuid
import pandas.io.sql as psql


In [99]:
from datetime import datetime
from typing import List, Optional
#from pydantic import BaseModel
import json
import psycopg2.extras

# call it in any place of your program
# before working with UUID objects in PostgreSQL
psycopg2.extras.register_uuid()

<psycopg2._psycopg.type 'UUID' at 0x7f8921766890>

In [123]:
def get_dataframe_from_sql(table_name: str, conn_params: dict):
    connection = psycopg2.connect(**conn_params)
    cursor = connection.cursor()
    df = psql.read_sql(f"Select * from {table_name}", connection)
    return df

In [124]:
# Connect to PostgreSQL
conn_params = {
    "host":"localhost",
    "database":"mydatabase",
    "user":"myuser",
    "password":"mypassword"

}

table_name = "propertypricetable"
df_fetch = get_dataframe_from_sql(table_name, conn_params)
df_fetch.head()

/var/folders/rj/64nmqpkj7j1dz_vk_9vdrqfw0000gn/T/ipykernel_39046/4176278397.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(f"Select * from {table_name}", connection)


,id,area,floor_range,no_of_units,contract_date,type_of_sale,price,property_type,district,type_of_area,tenure,nett_price,project,street,market_segment,x,y
0,bd8e964b-8b01-45ca-9cd5-d5810b9588d4,159.3,-,1,2020-03-01,3,2630000.0,Terrace,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.198897,30589.107079
1,54d728f7-df61-42fc-ab22-f7ef08bbe7dd,186.0,-,1,2021-03-01,3,1880000.0,Strata Terrace,05,Strata,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.198897,30589.107079
2,b5190668-75e4-49e5-9fc4-81b504d8d35a,257.0,-,1,2022-05-01,3,4600000.0,Terrace,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.198897,30589.107079
3,d46fe197-3c71-4278-83ed-51bb2816db8e,524.3,-,1,2020-02-01,3,5500000.0,Semi-detached,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,ZEHNDER ROAD,RCR,NaN,NaN
4,a825db95-5239-4bd9-8e19-f21bb265d42b,308.0,-,1,2021-09-01,3,5200000.0,Semi-detached,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,ZEHNDER ROAD,RCR,NaN,NaN


connection = psycopg2.connect(**conn_params)
cursor = connection.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)
cursor.close()
connection.close()

In [126]:
df_ec = df_fetch.loc[(df_fetch['property_type']=='Executive Condominium')&(df_fetch['type_of_sale']=='3')].sort_values(by=['contract_date'])

In [127]:
df_ec.head(2)

,id,area,floor_range,no_of_units,contract_date,type_of_sale,price,property_type,district,type_of_area,tenure,nett_price,project,street,market_segment,x,y
126465,70233109-b115-470a-8598-c6ac7fac4400,120.0,11-15,1,2019-09-01,3,853000.0,Executive Condominium,23,Strata,99 yrs lease commencing from 1997,NaN,YEW MEI GREEN,CHOA CHU KANG NORTH 6,OCR,18819.03588,41984.19449
124157,836f7655-2f22-4c9f-8461-e09aa66dbd46,90.0,11-15,1,2019-09-01,3,875000.0,Executive Condominium,23,Strata,99 yrs lease commencing from 2011,NaN,BLOSSOM RESIDENCES,SEGAR ROAD,OCR,20983.51444,41310.75604


In [128]:
df_ec.tail(2)

,id,area,floor_range,no_of_units,contract_date,type_of_sale,price,property_type,district,type_of_area,tenure,nett_price,project,street,market_segment,x,y
99951,edf3f4fe-d245-46f0-b823-31df5c118311,99.0,11-15,1,2024-08-01,3,1780000.0,Executive Condominium,19,Strata,99 yrs lease commencing from 2010,NaN,ESPARINA RESIDENCES,COMPASSVALE BOW,OCR,34460.87998,40428.28628
99769,242083df-6b08-4c08-83f6-5a1cfd6345f8,127.0,11-15,1,2024-08-01,3,1530000.0,Executive Condominium,19,Strata,99 yrs lease commencing from 1997,NaN,THE RIVERVALE,RIVERVALE LINK,OCR,35185.81787,40330.00568


In [130]:
df_ec['no_of_units'].unique()

array([1])

In [131]:
df_ec['type_of_area'].unique()

array(['Strata'], dtype=object)

In [133]:
columns_to_drop = ['no_of_units','type_of_area']
df_ec.drop(columns=columns_to_drop,inplace=True)

In [132]:
df_ec['tenure'].unique()

array(['99 yrs lease commencing from 1997',
       '99 yrs lease commencing from 2011',
       '99 yrs lease commencing from 2010',
       '99 yrs lease commencing from 1996',
       '99 yrs lease commencing from 2000',
       '99 yrs lease commencing from 2003',
       '99 yrs lease commencing from 2001',
       '99 yrs lease commencing from 2004',
       '99 yrs lease commencing from 2002',
       '99 yrs lease commencing from 2012',
       '99 yrs lease commencing from 2013',
       '99 yrs lease commencing from 2015',
       '99 yrs lease commencing from 2014',
       '99 yrs lease commencing from 2016',
       '99 yrs lease commencing from 2018'], dtype=object)

In [135]:
df_ec['tenure_start'] = df_ec['tenure'].apply(lambda x: int(x.split(' ')[-1]))
print(df_ec['tenure_start'].unique())
df_ec['building_age'] = df_ec['contract_date'].dt.year - df_ec['tenure_start']
df_ec.head()

[1997 2011 2010 1996 2000 2003 2001 2004 2002 2012 2013 2015 2014 2016
 2018]


,id,area,floor_range,contract_date,type_of_sale,price,property_type,district,tenure,nett_price,project,street,market_segment,x,y,tenure_start,building_age
126465,70233109-b115-470a-8598-c6ac7fac4400,120.0,11-15,2019-09-01,3,853000.0,Executive Condominium,23,99 yrs lease commencing from 1997,NaN,YEW MEI GREEN,CHOA CHU KANG NORTH 6,OCR,18819.03588,41984.19449,1997,22
124157,836f7655-2f22-4c9f-8461-e09aa66dbd46,90.0,11-15,2019-09-01,3,875000.0,Executive Condominium,23,99 yrs lease commencing from 2011,NaN,BLOSSOM RESIDENCES,SEGAR ROAD,OCR,20983.51444,41310.75604,2011,8
66715,41135f16-58a7-45a5-8c51-b273d7659119,129.0,11-15,2019-09-01,3,950000.0,Executive Condominium,18,99 yrs lease commencing from 1997,NaN,PINEVALE,TAMPINES STREET 73,OCR,39121.60695,37836.26604,1997,22
124163,0c5e0bef-b54a-4ed2-8b89-c5ac77deab25,98.0,06-10,2019-09-01,3,825000.0,Executive Condominium,23,99 yrs lease commencing from 2011,NaN,BLOSSOM RESIDENCES,SEGAR ROAD,OCR,20983.51444,41310.75604,2011,8
124166,4c3d88bb-2975-4ebf-b221-5efb82e2f45b,98.0,01-05,2019-09-01,3,890000.0,Executive Condominium,23,99 yrs lease commencing from 2011,NaN,BLOSSOM RESIDENCES,SEGAR ROAD,OCR,20983.51444,41310.75604,2011,8


In [114]:
#